# 🚀 AGI Core Hunter - MDL vs OOD Experiment

Bu notebook AGI Core Hunter projesinin **MDL vs OOD Generalization** deneyini Google Colab'da çalıştırır.

**Hipotez:** Yüksek β değerine sahip ajanlar daha iyi OOD genelleme performansı gösterecek.

**⚠️ Önemli:** Runtime → Change runtime type → GPU seçin!

In [ ]:
# 🖥️ GPU Kontrolü ve Proje Kurulumu
!nvidia-smi
print("\n🔄 Proje klonlanıyor...")
!git clone https://github.com/inkbytefo/agi-core-hunter.git
%cd agi-core-hunter

# Update pip
!pip install --upgrade pip -q

# Install the latest stable version of JAX with CUDA support
!pip install "jax[cuda]" -q

print("✅ JAX CUDA 12 installed!")
!pip install flax optax chex wandb tqdm matplotlib seaborn pandas numpy 

import jax
print(f"JAX version: {jax.__version__}")
print(f"Available devices: {jax.devices()}")

!nvcc --version

import jax
print(f"JAX version: {jax.__version__}")
print(f"Available devices: {jax.devices()}")



In [ ]:
# 📊 Wandb Login
import wandb
print("🔑 Wandb API key gerekli: https://wandb.ai/authorize")
wandb.login()

# Alternatif offline mode:
# import os
# os.environ['WANDB_MODE'] = 'offline'

In [ ]:
# 📋 Deney Konfigürasyonu
%cd experiments/01_mdl_vs_ood
import json
with open('manifest.json', 'r') as f:
    config = json.load(f)

print("🧪 DENEY DETAYLARI:")
print(f"Hipotez: {config['experiment']['hypothesis']}")
print(f"Episode: {config['training']['total_episodes']}")
print("Ajanlar:")
for agent in config['agents']:
    print(f"  {agent['name']}: β={agent['config']['beta']}")
print(f"OOD testleri: {len(config['evaluation']['ood_tests'])}")

In [ ]:
# 🧪 DENEY ÇALIŞTIR
import time
start_time = time.time()

print("🚀 MDL vs OOD deneyi başlıyor... (30-45 dakika)")
!python train.py

duration = time.time() - start_time
print(f"\n🎉 Deney tamamlandı! Süre: {duration/60:.1f} dakika")

In [ ]:
# 📊 SONUÇ ANALİZİ
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Sonuçları yükle
with open('results.json', 'r') as f:
    results = json.load(f)

# Veri hazırla
data = []
for agent, agent_data in results.items():
    beta = agent_data['agent_config']['beta']
    standard_success = agent_data['standard_success_rate']
    ood_ratios = [v for k, v in agent_data['ood_performance_ratios'].items() 
                 if k.endswith('_performance_ratio')]
    avg_ood_ratio = np.mean(ood_ratios)
    
    data.append({
        'Agent': agent,
        'Beta': beta,
        'Standard_Success': standard_success,
        'Avg_OOD_Ratio': avg_ood_ratio
    })

df = pd.DataFrame(data).sort_values('Beta')
print("📋 SONUÇLAR:")
print(df.round(3))

# Görselleştirme
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Beta vs OOD Performance
ax1.scatter(df['Beta'], df['Avg_OOD_Ratio'], s=100, c='red')
for i, agent in enumerate(df['Agent']):
    ax1.annotate(agent.replace('MDL_', ''), 
                (df['Beta'].iloc[i], df['Avg_OOD_Ratio'].iloc[i]))
ax1.set_xlabel('Beta (Compression Pressure)')
ax1.set_ylabel('Average OOD Performance Ratio')
ax1.set_title('🎯 MDL Hypothesis Test')
ax1.grid(True, alpha=0.3)

# Performance Comparison
x_pos = range(len(df))
ax2.bar([x-0.2 for x in x_pos], df['Standard_Success'], 0.4, 
        label='Standard', alpha=0.7)
ax2.bar([x+0.2 for x in x_pos], df['Avg_OOD_Ratio'], 0.4, 
        label='OOD Ratio', alpha=0.7)
ax2.set_xlabel('Agents')
ax2.set_ylabel('Performance')
ax2.set_title('📊 Performance Comparison')
ax2.set_xticks(x_pos)
ax2.set_xticklabels([a.replace('MDL_', '') for a in df['Agent']])
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Korelasyon analizi
correlation = np.corrcoef(df['Beta'], df['Avg_OOD_Ratio'])[0, 1]
print(f"\n📈 Beta-OOD Korelasyonu: {correlation:.3f}")

if correlation > 0.3:
    print("✅ HİPOTEZ DESTEKLENDİ!")
    print("Yüksek β daha iyi OOD genelleme sağlıyor.")
elif correlation < -0.3:
    print("❌ HİPOTEZ REDDEDİLDİ!")
    print("Yüksek β OOD performansını kötüleştiriyor.")
else:
    print("❓ HİPOTEZ BELİRSİZ")
    print("β ve OOD performansı arasında net ilişki yok.")

In [ ]:
# 💾 Sonuçları Kaydet
from google.colab import files

# Sonuç dosyalarını hazırla
!mkdir -p results_package
!cp results.json results_package/
!cp manifest.json results_package/config.json

# Özet rapor oluştur
summary = {
    "experiment": "MDL vs OOD Generalization",
    "completion_time": time.strftime("%Y-%m-%d %H:%M:%S"),
    "hypothesis_result": "Supported" if correlation > 0.3 else "Rejected" if correlation < -0.3 else "Inconclusive",
    "correlation": float(correlation),
    "duration_minutes": duration/60,
    "agent_results": df.to_dict('records')
}

with open('results_package/summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

# Zip olarak paketla ve indir
!zip -r mdl_ood_results.zip results_package/
files.download('mdl_ood_results.zip')

print("✅ Sonuçlar kaydedildi ve indirildi!")